In [ ]:
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
import praw
import prawcore
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm
import pickle
from random import sample, choice
import concurrent.futures

# List of subreddits to scrape

this list was build from https://anvaka.github.io/redsim. Can be used to expand the list of favourable subreddits.

takeing these for now

In [ ]:
subs = ["Sexpolls",
"sexpositions",
"Sexconfessional",
"penissize",
"masturbation",
"AskRedditNSFW",
"sexstories",
"SexFantasies",
"sexconfession",
"askwoman_aboutsex",
"sextips",
"sexualhealth",
"SexPositive",
"DirtyConfession",
"Puberty",
"NSFWIAMA",
"sexover30",
"SexToys",
"sexquestions",
"deepvaginaproblems",
"kegels",
"sexeducation",
"ColoredLang",
"masterbationstories",
"RedditAfterDark",
"Threesome_advice",
]

# "NSFWIAMA" -> special one (TODO f)

# Scrap these Subreddits

In [ ]:
from utils import scrap_subreddit, init_praw_reddit, save_to_huggingface


In [ ]:
reddit = init_praw_reddit()

# loop through subreddits and save to an intermediate folder
for sub in tqdm(subs):
    try:
        df = scrape_subreddit(sub, reddit)
        if df is not None:
            file_name = f"dataframes/{sub}.csv"
            df.to_csv(file_name, index=False)
            print("subreddit saved to: ", file_name)
    except Exception as e:
        logger.error(f"Error scraping {sub}: {e}")


In [ ]:
import os

# clean the results
files = os.listdir("dataframes/")
dfs = []
for file in files:
    df = pd.read_csv(f"dataframes/{file}")
    dfs.append(df[df['is_question'] & df['is_self']])
    
cleaned_df = pd.concat(dfs)
print(cleaned_df.shape, cleaned_df['subreddit'].value_counts())

In [46]:
save_to_huggingface(cleaned_df, name="jjmachan/NSFW-questions-inter-cleaned_df")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jjmachan/.cache/huggingface/datasets/json/default-9433ebf74f6f4bdb/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/13 [00:00<?, ?ba/s]

# Filter with prosocial

<explain what is done @shahul>

# Get Comments

In [ ]:
get_comments(cleaned_df['post_id']

In [42]:
nsfw_with_comments = pd.read_csv("df_with_comments.csv").drop_duplicates(subset=["post_id"])
nsfw = pd.read_csv("nsfw.csv").drop_duplicates(subset=["post_id"])
nsfw_final = pd.merge(nsfw.drop(columns=[f"C{i+1}" for i in range(5)]), nsfw_with_comments, on="post_id")
print(nsfw.shape, nsfw_with_comments.shape, nsfw_final.shape)

(12269, 14) (1442, 6) (1442, 14)


In [44]:
nsfw_final.sample(15)

,title,subreddit,post_id,score,link_flair_text,is_self,over_18,upvote_ratio,is_question,C1,C2,C3,C4,C5
920,How safe are condoms?,sexeducation,o610pr,12,NaN,True,False,1.00,True,"Hey love, it's great that you're being careful...","Well, I’ve never had sex, so I can’t say for s...",Hi Ppweewee69!\n\nWe've got the effectiveness ...,Condoms are over 98% effective when used CORRE...,"They are mainly safe, just have to watch a tut..."
477,how do foot fetishes work for women?,sexquestions,11euqxe,1,Kinks/Fetishes,True,True,1.00,True,"How do they work for men? Not being sassy, I m...",NaN,NaN,NaN,NaN
248,Would you be a sugar daddy without having sex ...,AskRedditNSFW,118q7zy,2,NaN,True,True,1.00,True,That she would dress the way I wanted. That's ...,Hell nah,Yes! She would need to come over and hang out ...,Terms,NaN
305,Men and women : Since rape is an act of violen...,AskRedditNSFW,lr3ssd,25,NaN,True,True,0.90,True,"The other comments answered this pretty well, ...",The idea behind rape fantasies is that of powe...,Thank you for the explanations. I now underst...,It's about control as a dynamic of BDSM. Rape ...,"it doesn't have to be about hate, it can also ..."
1144,Coping through birthday?,depression,11hmrxc,1,NaN,True,False,1.00,True,NaN,NaN,NaN,NaN,NaN
669,"Article: ""Why Are Young People Having So Littl...",sexover30,9y3ris,123,NaN,True,True,0.94,True,I think how much human interaction has been re...,"Fascinating read, thanks so much for posting! ...","I read this yesterday, and I’ve been mulling o...","The article weighs in at almost 15,000 words. ...",I think kids and work play far more into it th...
899,Why do we have kinks and fetishes,sexeducation,czdanh,20,NaN,True,False,1.00,True,There is some science on why but I dont know a...,That's a great question. Would love to read up...,NaN,NaN,NaN
336,is having a rape fantasy bad??,SexFantasies,or04sb,19,NaN,True,True,0.95,True,"No, it isn’t ….🥵","I don't think it's bad, I have this fantasy al...","its a cnc kink, very common","No, not at all!",I am f also and have that fantasy! <3 You're n...
189,What are your fantasies?,AskRedditNSFW,11hf12m,2,NaN,True,True,1.00,True,Being a nude competitive swimmer or a nude art...,I absolutely love being a free use fuck toy!,Have sex with multiple men as it's steamed glo...,I would be smitten with a room full of men pas...,To receive boob pics from women online 🤣😎
902,Can I be pregnant?,sexeducation,mtdnqf,16,NaN,True,False,0.94,True,"It sounds like you were extra careful, if you ...",Pull out is not a safe way since som sperm cum...,It doesn't sound like you have anything to wor...,NaN,NaN


# Save to Huggingface

In [40]:
from utils import save_to_huggingface

save_to_huggingface(nsfw_final, name="jjmachan/NSFW-questions")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /home/jjmachan/.cache/huggingface/datasets/json/default-424060f196a2282b/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]